In [4]:
import numpy as np
import sys
#np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_PFCI import PFHamiltonianGenerator
from helper_PFCI import Determinant
from helper_cqed_rhf import cqed_rhf
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)
from psi4.driver.procrouting.response.scf_response import tdscf_excitations

In [5]:
# options for mgf
mol_str = """
Mg
H 1 2.2
symmetry c1
1 1
"""

options_dict = {
    "basis": "cc-pVDZ",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
    "save_jk" : True
}


psi4.set_options(options_dict)
mol = psi4.geometry(mol_str)


rhf_energy, wfn = psi4.energy("scf/cc-pVDZ", molecule=mol, return_wfn=True)
    
# calculate the excited-state energies and save them to a dictionary called 'res'
res = tdscf_excitations(wfn, states=5, triplets = "NONE", tda=True)


# parse res for excitation energies
psi4_excitation_e = [r["EXCITATION ENERGY"] for r in res]

psi4_tdm = [r["ELECTRIC DIPOLE TRANSITION MOMENT (LEN)"] for r in res]


print(rhf_energy)
print(rhf_energy + psi4_excitation_e[0])

-199.86395910419137
-199.69011028328705


In [6]:
print(psi4_tdm[0])

[-6.93218490e-16 -1.77990759e-15  2.33258251e+00]


In [11]:
mol_str = """
Mg
H 1 2.2
symmetry c1
1 1
"""

options_dict = {
    "basis": "cc-pVDZ",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}

cavity_dict = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0]),
    'ci_level' : 'cis',
    'full_diagonalization' : True,
    'no_cavity' : True
}

mol = psi4.geometry(mol_str)

psi4.set_options(options_dict)

test_pf = PFHamiltonianGenerator(
    mol_str,
    options_dict,
    cavity_dict
)




Start SCF iterations:

Canonical RHF One-electron energy = -281.6658725092592590
CQED-RHF One-electron energy      = -281.6658725092592590
Nuclear repulsion energy          = 2.8864211491090908
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -199.8639591041915651   dE = -1.99864E+02   dRMS = 1.51684E-12
SCF Iteration   2: Energy = -199.8639591041911103   dE =  4.54747E-13   dRMS = 2.67192E-13
Total time for SCF iterations: 0.002 seconds 

QED-RHF   energy: -199.86395910 hartree
Psi4  SCF energy: -199.86395910 hartree
 Completed QED-RHF in 0.7523939609527588 seconds
 Completed 1HSO Build in 0.0002620220184326172 seconds
 Completed ERI Build in 0.501763105392456 seconds 
 Completed 2D build in 0.040303707122802734 seconds
 Completed 1G build in 8.606910705566406e-05 seconds
 Completed the Dipole Matrix Build in 0.0008351802825927734 seconds
 Completed determinant list in 0.006983041763305664 seconds 
 Completed constant offset matrix in 0.0001609325408

In [12]:
#test_pf.cis_c = np.copy(test_pf.CIvecs)
#test_pf.cis_e = np.copy(test_pf.CIeigs)
#test_pf.CISnumDets = len(test_pf.CIeigs) // 2
_CI_No_Cav = test_pf.ApDmatrix + test_pf.Enuc_so + test_pf.dc_so
test_pf.cis_e, test_pf.cis_c = np.linalg.eigh(_CI_No_Cav)

_numDets = len(test_pf.cis_e)

test_pf.CIvecs = np.copy(test_pf.cis_c)
test_pf.CIeigs = np.copy(test_pf.cis_e)
test_pf.MU_X = np.copy(test_pf.MU_X[:_numDets, :_numDets])
test_pf.MU_Y = np.copy(test_pf.MU_Y[:_numDets, :_numDets])
test_pf.MU_Z = np.copy(test_pf.MU_Z[:_numDets, :_numDets])


In [97]:
N_el = 2
N_ph = 3
_singlets = [0]

mu_r_sing = np.zeros((N_el,N_el,3))
zero_vec = np.array([0., 0., 0.])
# we want to find the first 20 states that are coupled to the ground state through a dipole transition
_num_kets = 0
_ket_idx = 1
_sing_idx = 1
while _num_kets < N_el - 1:
    _tmp_mu = test_pf.compute_dipole_moment(0, _ket_idx)
    if np.allclose(zero_vec, _tmp_mu):
        _ket_idx += 1
    else:
        mu_r_sing[0, _sing_idx, :] = _tmp_mu
        _singlets.append(_ket_idx)
        _ket_idx += 1
        _num_kets += 1
        _sing_idx += 1
        
print(_singlets)
    
E_R_sing = test_pf.cis_e[_singlets]
print(E_R_sing)

# now get all dipole moments
for i in range(0, len(_singlets)):
    a = _singlets[i]
    for j in range(0, len(_singlets)):
        b = _singlets[j]
        mu_r_sing[i, j, :] = test_pf.compute_dipole_moment(a, b) + test_pf.mu_nuc * (a==b)
        
print(mu_r_sing)

[0, 2]
[-199.8639591  -199.69011028]
[[[ 4.32586196e-15 -1.08762579e-15 -1.50279429e+00]
  [ 4.15864794e-13 -8.54989970e-14 -2.33258235e+00]]

 [[ 4.15864794e-13 -8.54989970e-14 -2.33258235e+00]
  [-6.62427709e-13  1.50558858e-13  2.07947966e+00]]]


\begin{equation}
\hat{H}_{\alpha n, \beta m} = \left( E_{\alpha}(R) + n \omega \right) \delta_{\alpha \beta} \delta_{nm} + \sqrt{\frac{\omega}{2}} \lambda \cdot \mu_{\alpha \beta}(R) \left(\sqrt{n} \delta_{n,m-1} + \sqrt{n+1} \delta_{n,m+1} \right)  \\
+\frac{1}{2} \sum_{\gamma = 1}^N \lambda \cdot \mu_{\alpha, \gamma}(R) \lambda \cdot \mu_{\gamma, \beta} \delta_{nm}
\end{equation}


In [98]:
def build_pf_hamiltonian(n_el, n_ph, E_R, omega, lamvec, mu):
    """
    Given an array of n_el E_R values and an n_ph states with fundamental energy omega
    build the PF Hamiltonian
    
    n_el : int
        the number of electronic states (n_el = 1 means only ground-state)
    
    n_ph : int
        the number of photon occupation states (n_ph = 1 means only the |0> state)
    
    E_R : np.array of floats
        the electronic energies
        
    omega : float
        the energy of the photonic mode
        
    lamvec : np.array of floats
        the lambda vector
        
    mu : (n_el x n_el x 3) np.array of floats 
        mu[i, j, k] is the kth cartesian component of the dipole moment expectation value between 
        state i and state j
    
    """
    H_PF = np.zeros((n_el * n_ph, n_el * n_ph))
    
    # take care of the diagonals first
    # bare electronic and photonic energy
    for n in range(n_ph):
        for a in range(n_el):
            na = n * n_el + a
            H_PF[na,na] = E_R[a] + n * omega
        
    # diagonal dipole self energy
    for n in range(n_ph):
        for a in range(n_el):
            na = n * n_el + a
            for g in range(n_el):
                H_PF[na,na] += 0.5 * np.dot(lamvec, mu[a,g,:]) * np.dot(lamvec, mu[g,a,:])
            
    # off-diagonal dipole self energy
    for n in range(n_ph):
        for a in range(n_el):
            na = n * n_el + a
            for b in range(n_el):
                nb = n * n_el + b
                for g in range(n_el):
                    H_PF[na, nb] += 0.5 * np.dot(lamvec, mu[a,g,:]) * np.dot(lamvec, mu[g, b, :])
                
    # off-diagonal bilinear coupling
    for n in range(n_ph):
        for a in range(n_el):
            na = n * n_el + a
            for m in range(n_ph):
                for b in range(n_el):
                    nb = m * n_el + b
                    H_PF[na,nb] += np.sqrt(omega / 2) * np.dot(lamvec, mu[a,b,:]) * (np.sqrt(n) * n == (m-1) + np.sqrt(n+1) * n == (m+1))
    
    return H_PF
                
            
    
        
    
    

In [99]:
omega_cav = 4.75 / psi4.constants.Hartree_energy_in_eV
lambda_vector = np.array([0, 0, 0.00125])

In [100]:
_HPF = build_pf_hamiltonian(N_el, N_ph, E_R_sing, omega_cav, lambda_vector, mu_r_sing)

In [101]:
print(_HPF)

[[-1.99863947e+02 -1.05091103e-06  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.05091103e-06 -1.99690095e+02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.99689388e+02 -1.05091103e-06
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.05091103e-06 -1.99515536e+02
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -1.99514829e+02 -1.05091103e-06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -1.05091103e-06 -1.99340976e+02]]


In [96]:
pf_e, pf_c = np.linalg.eigh(_HPF)
print(pf_e)

[-199.86394707 -199.69009503 -199.68938779 -199.51553574]


In [86]:
print(pf_e[2]-pf_e[0])
#print(pf_e[3])
print(omega_cav)
#print(pf_e[1]-pf_e[0])
#print(pf_e[2]-pf_e[1])

0.1745592817840702
0.1745592817840596


In [55]:
expected_mghp_g_e = -199.86358254419457
expected_mghp_lp_e = -199.69776087489558
expected_mghp_up_e = -199.68066502792058

print(expected_mghp_lp_e-expected_mghp_g_e)
print(expected_mghp_up_e-expected_mghp_lp_e)

0.16582166929899245
0.01709584697499622
